# Init 

In [1]:
import sys
import os
import json
from pathlib import Path
import tensorflow as tf
from keras import backend as K

from keras.optimizers.legacy import Adam, SGD

from utils.loss import *
from utils.metrics import *
from utils.dataset import *
from keras.layers import *
from keras.models import *

base_path = '/Users/lorenzofederici/Univpm/Tesi/LUS-multitask-learning'
if base_path not in sys.path:
    sys.path.append(base_path)

# import custom lib
from src.utils.dataset import *
from src.experiment import Experiment
from models import resnet18 as r

gpu = len(tf.config.list_physical_devices('GPU'))>0
if gpu:
    tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0],tf.config.list_physical_devices('CPU')[0]])
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
    print("---> GPU is available <---")

exps_path = '/Users/lorenzofederici/Univpm/Tesi/LUS-multitask-learning/config/params.json'

with open(exps_path) as f:
    configs = json.load(f)

---> GPU is available <---


In [2]:
experiment = Experiment(base_path, output_mode= (True,True))
# TODO: chiedi a edo di smote

In [3]:
idx = 0
configs_general = configs['SETTING']
configs_exps    = configs['EXPS'][idx]

for config in configs_exps:
    experiment.build_experiment(configs_general, config)

print('>>Setting: ', configs_general)
print('>>Experiment: ', configs_exps)

experiment.split_dataset()
experiment.generate_split_charts('pierclass')



------------------------------------------------------------------------------------------
>> EXPERIMENT: exp_classification_ResNet_BS16_EP2_OPTSGD_LR0.01_AUGTrue__DT2211_1226 <<
------------------------------------------------------------------------------------------
230 videos (2154 frames) loaded from cached data.


------------------------------------------------------------------------------------------
>> EXPERIMENT: exp_classification_ResNet_BS16_EP2_OPTSGD_LR0.01_AUGTrue__DT2211_1226 <<
------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------
>> EXPERIMENT: exp_classification_ResNet_BS16_EP2_OPTSGD_LR0.01_AUGTrue__DT2211_1226 <<
------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------
>> EXPERIMENT: exp_classification_ResNet_

2023-11-22 12:26:31.635252: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-22 12:26:31.635296: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-22 12:26:31.635302: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-22 12:26:31.635351: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-22 12:26:31.635378: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
K.clear_session
model = experiment.build_model()
model   = experiment.compile_model(model)



>> BUILDING MODEL:
	Task --> classification
	Model --> ResNet_50
	Batch size --> 16
	Optimizer --> SGD lr:0.01
	Epoch --> 2
	Model --> True
	Dropout --> 0.0
	Trasfer learning Active from IMAGENET
	Freeze --> enc_2


In [5]:
history = experiment.train_model(model)



------------------------------------------------------------
		   ~~~~~~ TRAINING ~~~~~
------------------------------------------------------------
Epoch 1/2


2023-11-22 12:26:40.859692: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


68/68 [==============================] - 80s 1s/step - loss: 1.2095 - accuracy: 0.4681 - val_loss: 1.7927 - val_accuracy: 0.3086 - lr: 0.0100
Epoch 2/2
68/68 [==============================] - 70s 1s/step - loss: 0.7790 - accuracy: 0.6642 - val_loss: 1.8255 - val_accuracy: 0.2026 - lr: 0.0100


In [ ]:
eval = experiment.evaluation_model(model)

In [ ]:
experiment.nn_train_graphs(history)

# GridSearch

In [ ]:
from tensorflow import keras
from utils.loss import categorical_crossentropy
from keras.optimizers.legacy import Adam, SGD
from models.resnet18 import resnet18

In [ ]:
import keras_tuner
from keras_tuner.tuners import BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters


In [ ]:
# Definire la funzione del modello
def build_model(hp):
    model = resnet18((224, 224, 3), num_class=4)

    hp_opt = Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(
        optimizer    = hp_opt,
        loss         = categorical_crossentropy,
        metrics      = 'accuracy')
    return model


In [ ]:
K.clear_session

tuner = BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=5,  # Numero totale di prove di addestramento del modello
    num_initial_points=2,  # Numero di punti iniziali per la ricerca casuale
    directory=f'{base_path}/grid_result',  # Directory per salvare i risultati del tuner
    project_name='my_project')  # Nome del progetto

In [ ]:
batch_size = 16
train_samples = len(experiment.train_subset_idxs)
train_steps = train_samples // batch_size
if train_samples % batch_size != 0:
    train_steps += 1

val_samples = len(experiment.val_subset_idxs)
val_steps = val_samples // batch_size
if val_samples % batch_size != 0:
    val_steps += 1

tuner.search(
    experiment.train_ds,
    epochs           = 10,
    steps_per_epoch  = train_steps,
    validation_data  = experiment.val_ds,
    class_weight     = experiment.class_weight_dicts[0],
    validation_steps = val_steps,
    workers = 8
)

In [ ]:
# Ottenere il miglior modello
best_model = tuner.get_best_models(num_models=1)[0]

# Stampa delle migliori iperparametri
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")